In [1]:
import digitalhub as dh

project = dh.get_or_create_project("llm-ci")

In [2]:
llm_function = project.new_function("llm_classification",
                                    kind="huggingfaceserve",
                                    model_name="mymodel",
                                    path="huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [3]:
llm_run = llm_function.run(action="serve", profile="template-a100")

Wait for job to be in 'running' state. One can check the status of 'Run' from console or by typing the following command

In [ ]:
llm_run.get_state()

Once running, make API call to the serve job.

In [9]:
model_name = "mymodel"
json = {
    "inputs": [
        {
            "name": "input-0",
            "shape": [2],
            "datatype": "BYTES",
            "data": ["Hello, my dog is cute", "I am feeling sad"],
        },
    ]
}

In [5]:
llm_run.refresh()

{'kind': 'huggingfaceserve+run', 'metadata': {'project': 'llm-ci', 'name': '47f6687c-44f1-415d-90d3-f7464bc351b9', 'created': '2024-10-29T11:49:03.487Z', 'updated': '2024-10-29T11:50:04.117Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'huggingfaceserve+serve://llm-ci/llm_classification:7713de1f-465d-4a53-954a-bf89170cdafc', 'local_execution': False, 'function': 'huggingfaceserve://llm-ci/llm_classification:7713de1f-465d-4a53-954a-bf89170cdafc', 'profile': 'template-a100', 'path': 'huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english', 'model_name': 'mymodel', 'service_type': 'NodePort'}, 'status': {'state': 'RUNNING', 'message': 'deployment d-huggingfaceserveserve-47f6687c-44f1-415d-90d3-f7464bc351b9 created, service s-huggingfaceserveserve-47f6687c-44f1-415d-90d3-f7464bc351b9 created', 'transitions': [{'status': 'RUNNING', 'message': 'deployment d-huggingfaceserveserve-47f6687c-44f1-415d-90d3-f7464bc351b9 created, service s

In [10]:
llm_run.invoke(model_name=model_name, json=json).json()

{'model_name': 'mymodel',
 'model_version': None,
 'id': 'e5e22206-9911-4b06-b8d0-1e34bf3d98eb',
 'parameters': None,
 'outputs': [{'name': 'output-0',
   'shape': [2],
   'datatype': 'INT64',
   'parameters': None,
   'data': [1, 0]}]}

In [11]:
import os
new_folder = 'src'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

In [12]:
%%writefile "src/llm_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(path): 
    with pipeline_context() as pc:
         serve = pc.step(name="serve", function="llm_classification", action="serve", inputs={"path":path})

Writing src/llm_pipeline.py


In [13]:
workflow = project.new_workflow(name="pipeline_llm", kind="kfp", code_src= "src/llm_pipeline.py", handler = "myhandler")

In [14]:
workflow.run(parameters={"path": 'huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english'})

{'kind': 'kfp+run', 'metadata': {'project': 'llm-ci', 'name': '4d9ed937-917e-42ed-bc7d-691444aadeb3', 'created': '2024-10-29T11:52:07.212Z', 'updated': '2024-10-29T11:52:07.228Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'kfp+pipeline://llm-ci/pipeline_llm:fa4619cb-84b3-4255-b457-b17e38195801', 'local_execution': False, 'function': 'kfp://llm-ci/pipeline_llm:fa4619cb-84b3-4255-b457-b17e38195801', 'source': {'source': 'src/llm_pipeline.py', 'handler': 'myhandler', 'base64': 'CmZyb20gZGlnaXRhbGh1Yl9ydW50aW1lX2tmcC5kc2wgaW1wb3J0IHBpcGVsaW5lX2NvbnRleHQKCmRlZiBteWhhbmRsZXIocGF0aCk6IAogICAgd2l0aCBwaXBlbGluZV9jb250ZXh0KCkgYXMgcGM6CiAgICAgICAgIHNlcnZlID0gcGMuc3RlcChuYW1lPSJzZXJ2ZSIsIGZ1bmN0aW9uPSJsbG1fY2xhc3NpZmljYXRpb24iLCBhY3Rpb249InNlcnZlIiwgaW5wdXRzPXsicGF0aCI6cGF0aH0pCg==', 'lang': 'python'}, 'inputs': {}, 'outputs': {}, 'parameters': {'path': 'huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english'}}, 'status': {'state': 'READY